In [9]:
from google.colab import drive
# with open('/gdrive/ColabML', 'w') as f:
#   f.write('Hello Google Drive!')
drive.mount('/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /gdrive


In [0]:
import re
import numpy as np
from sklearn.externals import joblib
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
import tensorflow as tf

In [0]:
class Training:
    def __init__(self):
        self.english_stop_words = stopwords.words('english')
        self.reviews_train = []
        self.reviews_test= []
        self.REPLACE_NO_SPACE = re.compile("(\.)|(\;)|(\:)|(\!)|(\')|(\?)|(\,)|(\")|(\()|(\))|(\[)|(\])")
        self.REPLACE_WITH_SPACE = re.compile("(<br\s*/><br\s*/>)|(\-)|(\/)")
    
    def split(self):
        print('Split')
        with tf.device('/device:GPU:0'):
            for a in open('/gdrive/My Drive/Colab_ML/senti/data/full_train.txt','r'):
                self.reviews_train.append(a.strip())
            for a in open('/gdrive/My Drive/Colab_ML/senti/data/full_test.txt','r'):
                self.reviews_test.append(a.strip())

    def remove_stop_words(self,corpus):
        removed_stop_words = []
        with tf.device('/device:GPU:0'):
            for review in corpus:
                removed_stop_words.append(
                    ' '.join([word for word in review.split() 
                            if word not in self.english_stop_words])
                )
        print('Done stop words')
        return removed_stop_words

    def get_stemmed_text(self,corpus):
        stemmer = PorterStemmer()
        print('Done stem')
        return [' '.join([stemmer.stem(word) for word in review.split()]) for review in corpus]
        
    def preprocess_reviews(self,reviews):
        with tf.device('/device:GPU:0'):
            reviews = [self.REPLACE_NO_SPACE.sub("", line.lower()) for line in reviews]
            reviews = [self.REPLACE_WITH_SPACE.sub(" ", line) for line in reviews]
            reviews = self.remove_stop_words(reviews)
            reviews = self.get_stemmed_text(reviews)
            print('Done preprocess')
        return reviews
    
    def mainTrain(self):
        print('Preprocess')
        
        reviews_train_cleaned = self.preprocess_reviews(self.reviews_train)
        reviews_test_cleaned = self.preprocess_reviews(self.reviews_test)
        
        with tf.device('/device:GPU:0'):
            ngram_vectorizer = CountVectorizer(binary=True, ngram_range=(1, 2))
            ngram_vectorizer.fit(reviews_train_cleaned)
            X = ngram_vectorizer.transform(reviews_train_cleaned)
            X_val = ngram_vectorizer.transform(reviews_test_cleaned)
            target = [1 if i < 12500 else 0 for i in range(25000)]
            X_train,X_test,y_train,y_test = train_test_split(X,target,train_size = 0.75)
            print('Starting training')
            # To find c
            '''
            for c in [0.01,0.05,0.25,0.5,1,1.5]:
                lr = LogisticRegression(C=c)
                lr.fit(X_train,y_train)
                print('Accuracy score for {}: {}'.format(c,accuracy_score(y_test,lr.predict(X_test))))
            '''
            c = 0.5
            lr = LogisticRegression(C=c)
            lr.fit(X_train,y_train)
        print('Accuracy score for {}: {}'.format(c,accuracy_score(y_test,lr.predict(X_test))))
        joblib.dump(lr,'model.sav')
        print('DONE')


In [28]:

a = Training()
a.split()
a.mainTrain()

Split
Preprocess
Done stop words
Done stem
Done preprocess
Done stop words
Done stem
Done preprocess


/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_split.py:2179: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


Starting training


/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


Accuracy score for 0.5: 0.88064
DONE
